# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# input csv data file
input_data_file = "../WeatherPy/output_data/cities.csv"
output_data_file = "../output_data/map.csv"

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df_1=pd.read_csv(input_data_file)
df_1

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Edson,53.5835,-116.4356,59.14,36,1,8.05,CA,1634854598
1,Guangzhou,23.1167,113.2500,60.94,71,95,12.75,CN,1634854289
2,Nikolskoye,59.7035,30.7861,53.87,85,100,16.40,RU,1634854598
3,Havre-St-Pierre,50.2334,-63.5986,50.00,87,1,4.61,CA,1634854598
4,Kuril'sk,45.2333,147.8833,41.11,84,100,4.29,RU,1634854599
...,...,...,...,...,...,...,...,...,...
570,‘Ewa Beach,21.3156,-158.0072,84.15,62,75,10.00,US,1634854787
571,Ulladulla,-35.3500,150.4667,66.09,84,32,2.15,AU,1634854788
572,Semey,50.4111,80.2275,43.86,83,100,16.55,KZ,1634854789
573,Kilindoni,-7.9139,39.6668,77.50,77,19,12.08,TZ,1634854789


In [3]:
df_1.dtypes

City                object
Latitude           float64
Longitude          float64
Max Temperature    float64
Humidity             int64
Cloudiness           int64
Wind Speed         float64
Country             object
Date                 int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = df_1[['Latitude', 'Longitude']]
weights = df_1['Humidity']

In [7]:
humidity_fig = gmaps.figure(map_type="HYBRID")
heatmap_layer=gmaps.heatmap_layer(locations,weights=weights)

# Addimg Heatmap layer to map
humidity_fig.add_layer(heatmap_layer)
heatmap_layer.max_intensity = df_1['Humidity'].max()
heatmap_layer.point_radius = 10
humidity_fig
plt.savefig("Images/Humidity_map.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
df_1

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Edson,53.5835,-116.4356,59.14,36,1,8.05,CA,1634854598
1,Guangzhou,23.1167,113.2500,60.94,71,95,12.75,CN,1634854289
2,Nikolskoye,59.7035,30.7861,53.87,85,100,16.40,RU,1634854598
3,Havre-St-Pierre,50.2334,-63.5986,50.00,87,1,4.61,CA,1634854598
4,Kuril'sk,45.2333,147.8833,41.11,84,100,4.29,RU,1634854599
...,...,...,...,...,...,...,...,...,...
570,‘Ewa Beach,21.3156,-158.0072,84.15,62,75,10.00,US,1634854787
571,Ulladulla,-35.3500,150.4667,66.09,84,32,2.15,AU,1634854788
572,Semey,50.4111,80.2275,43.86,83,100,16.55,KZ,1634854789
573,Kilindoni,-7.9139,39.6668,77.50,77,19,12.08,TZ,1634854789


In [9]:
#My weather conditions 
ideal_weather= df_1

# A max temperature lower than 80 degrees but higher than 70.
ideal_weather = ideal_weather.loc[(ideal_weather["Max Temperature"] < 80) & (ideal_weather["Max Temperature"] > 70)]

# Wind speed less than 10 mph.
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

ideal_weather

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
72,Venado Tuerto,-33.7456,-61.9688,77.79,32,0,8.37,AR,1634854620
85,Mar del Plata,-38.0023,-57.5575,75.22,90,0,3.00,AR,1634854625
110,Junín,-34.5838,-60.9433,78.98,34,0,5.91,AR,1634854450
161,Saint-Paul,-21.0096,55.2707,71.71,84,0,1.99,RE,1634854651
287,Saint-Pierre,-21.3393,55.4781,71.28,69,0,6.91,RE,1634854694
449,Alice Springs,-23.7000,133.8833,72.95,27,0,6.91,AU,1634854745
523,Campo Grande,-20.4428,-54.6464,78.35,44,0,5.75,BR,1634854773
538,Gangānagar,29.9167,73.8833,75.27,30,0,8.12,IN,1634854778


In [10]:
len(ideal_weather)

8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_weather 

# params dictionary to update each iteration
params = {
    "radius": 5000,"types": "lodging","key": g_key}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Nearest Hotel {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing Results.")
        
    print("------------")

Retrieving Results for Index 72: Venado Tuerto.
Nearest Hotel Hotel - Miro Park.
------------
Retrieving Results for Index 85: Mar del Plata.
Nearest Hotel Gran Hotel Mar del Plata.
------------
Retrieving Results for Index 110: Junín.
Nearest Hotel Hotel Central.
------------
Retrieving Results for Index 161: Saint-Paul.
Nearest Hotel Boucan Canot.
------------
Retrieving Results for Index 287: Saint-Pierre.
Nearest Hotel Lindsey Hôtel.
------------
Retrieving Results for Index 449: Alice Springs.
Nearest Hotel Desert Palms Alice Springs.
------------
Retrieving Results for Index 523: Campo Grande.
Nearest Hotel Hotel Harbor Self Buriti Suítes Campo Grande.
------------
Retrieving Results for Index 538: Gangānagar.
Nearest Hotel SPOT ON 72525 Govind Guest House.
------------


In [11]:
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Edson,53.5835,-116.4356,59.14,36,1,8.05,CA,1634854598,"Holiday Inn Express & Suites Edson, an IHG Hotel"
1,Guangzhou,23.1167,113.2500,60.94,71,95,12.75,CN,1634854289,Guangzhou Hotel
2,Nikolskoye,59.7035,30.7861,53.87,85,100,16.40,RU,1634854598,Tourist House - Sablino
3,Havre-St-Pierre,50.2334,-63.5986,50.00,87,1,4.61,CA,1634854598,Auberge Niapiskau
4,Kuril'sk,45.2333,147.8833,41.11,84,100,4.29,RU,1634854599,Hotel Ostrov
...,...,...,...,...,...,...,...,...,...,...
570,‘Ewa Beach,21.3156,-158.0072,84.15,62,75,10.00,US,1634854787,Uncle Junior Robert
571,Ulladulla,-35.3500,150.4667,66.09,84,32,2.15,AU,1634854788,Sandpiper Motel
572,Semey,50.4111,80.2275,43.86,83,100,16.55,KZ,1634854789,Grand Hotel Semey
573,Kilindoni,-7.9139,39.6668,77.50,77,19,12.08,TZ,1634854789,Bustani B&B


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Add marker layer ontop of heat map
hotel_details = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)
# Display figure
humidity_fig.add_layer(markers)
humidity_fig

Figure(layout=FigureLayout(height='420px'))